In [2]:
# Basic Phrases of Sign Language Data Collection using MediaPipe and OpenCV

import cv2
import mediapipe as mp
import pandas as pd

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,  # ✅ allow 2 hands
    min_detection_confidence=0.7
)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

data = []
labels = []

current_label = "Please"  # Change this when collecting other datasets

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    if results.multi_hand_landmarks:   # ✅ Only collect if hand(s) detected
        # Initialize with zeros (42 landmarks x 3 coords = 126 features)
        landmarks = [0.0] * (21 * 2 * 3)

        for hand_idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Extract 21 landmarks per hand
            for i, lm in enumerate(hand_landmarks.landmark):
                base_index = hand_idx * 63 + i * 3
                landmarks[base_index:base_index+3] = [lm.x, lm.y, lm.z]

        data.append(landmarks)
        labels.append(current_label)

    else:
        # ✅ Show message when no hand detected
        cv2.putText(frame, "No hand detected", (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Collecting Data", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to stop
        break

cap.release()
cv2.destroyAllWindows()

# Save collected dataset
df = pd.DataFrame(data)
df['label'] = labels
df.to_csv("hand_sign_data_please.csv", index=False)

print("✅ Data collection saved with 2-hand support")

c:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


✅ Data collection saved with 2-hand support


In [2]:
import pandas as pd
import glob
import os

# Folder containing all your datasets
DATA_DIR = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\basic-words"

# Load all CSV files in the folder
all_files = glob.glob(os.path.join(DATA_DIR, "*.csv"))

# Read and concatenate all into one dataframe
df_list = []
for file in all_files:
    temp_df = pd.read_csv(file)
    df_list.append(temp_df)

# Combine into one dataset
final_df = pd.concat(df_list, ignore_index=True)

# Save as one combined dataset
final_csv = os.path.join(DATA_DIR, "hand_sign_data_all.csv")
final_df.to_csv(final_csv, index=False)

print("✅ Combined dataset saved at:", final_csv)
print("Dataset shape:", final_df.shape)
print("Labels available:", final_df['label'].unique())

✅ Combined dataset saved at: C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\basic-words\hand_sign_data_all.csv
Dataset shape: (5858, 127)
Labels available: ['hello' 'i love you' 'Thank You' 'Yes' 'Please']


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle

# Load combined dataset
file_path = r"C:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\basic-words\hand_sign_data_all.csv"
df = pd.read_csv(file_path)

# Features (X) and labels (y)
X = df.drop("label", axis=1)
y = df["label"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train model
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Evaluate
print("Accuracy:", model.score(X_test, y_test))
print("Classes learned:", model.classes_)

# Save trained model
with open("sign_model.pkl", "wb") as f:
    pickle.dump(model, f)


Accuracy: 1.0
Classes learned: ['Please' 'Thank You' 'Yes' 'hello' 'i love you']


In [1]:
import cv2
import pickle
import numpy as np
import mediapipe as mp
from collections import deque

# Load trained model
with open("sign_model.pkl", "rb") as f:
    model = pickle.load(f)

# Mediapipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    # Initialize prediction text
    predicted_label = "No hands detected"

    if results.multi_hand_landmarks:
        landmarks = [0.0] * (21 * 2 * 3)

        for hand_idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Fill the correct slice of landmarks
            for i, lm in enumerate(hand_landmarks.landmark):
                base_index = hand_idx * 63 + i * 3
                landmarks[base_index:base_index+3] = [lm.x, lm.y, lm.z]

        # Convert to NumPy array
        landmarks = np.array(landmarks).reshape(1, -1)

        try:
            # Predict
            prediction = model.predict(landmarks)
            predicted_label = prediction[0]
        except Exception as e:
            predicted_label = f"Prediction error: {e}"

    # Show result on frame
    cv2.putText(frame, f"Predicted: {predicted_label}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Sign Language Recognition", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to quit
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\JamJayDatuin\Documents\Machine Learning Projects\TrainingAI-Models\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Ple